In [ ]:

import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import scipy.io as sio
import h5py
from mpl_toolkits.axes_grid1 import make_axes_locatable

file_path = "IMG_0195.mat"
varname = "pl"
output_gif = "animation_with_cursor.gif"
num_frames_wanted = 61
cmap_name = "gist_heat"
use_origin_lower = True
pmin, pmax = 2, 98
fps = 10
show_mean_std_band = True

if not os.path.exists(file_path):
    raise FileNotFoundError(file_path)

try:
    mat_data = sio.loadmat(file_path)
except NotImplementedError:
    with h5py.File(file_path, "r") as f:
        mat_data = {k: np.array(v) for k, v in f.items()}

if varname not in mat_data:
    raise KeyError(f"'{varname}' not in .mat")

image_stack = np.array(mat_data[varname])
if image_stack.ndim != 3:
    raise ValueError(f"expect 3D, got {image_stack.shape}")

H, W, N = image_stack.shape
frames = min(num_frames_wanted, N)

print(f"shape={image_stack.shape} | ใช้ {frames} ระนาบในการทำแอนิเมชัน")

vmin = np.percentile(image_stack[:, :, :frames], pmin)
vmax = np.percentile(image_stack[:, :, :frames], pmax)
slice_mean = np.array([image_stack[:, :, i].mean() for i in range(frames)])
slice_std  = np.array([image_stack[:, :, i].std()  for i in range(frames)])
peak_idx = int(np.argmax(slice_mean)) + 1  # 1-indexed

print(f"ช่วงคอนทราสต์: vmin={vmin:.2f}, vmax={vmax:.2f}")

plt.rcParams["figure.dpi"] = 110
plt.close("all")
fig = plt.figure(figsize=(12, 5), constrained_layout=True)
gs = fig.add_gridspec(1, 2, width_ratios=[1.15, 1.0], wspace=0.25)
ax_img  = fig.add_subplot(gs[0, 0])
ax_plot = fig.add_subplot(gs[0, 1])

im = ax_img.imshow(
    image_stack[:, :, 0],
    cmap=cmap_name, vmin=vmin, vmax=vmax,
    origin="lower" if use_origin_lower else "upper",
)
ax_img.set_title(f"{varname} | Slice 1 / {frames}")
ax_img.axis("off")

divider = make_axes_locatable(ax_img)
cax = divider.append_axes("right", size="4%", pad=0.04)
cb = fig.colorbar(im, cax=cax)
cb.set_label("Pixel Intensity (a.u.)")

x_idx = np.arange(1, frames + 1)
ax_plot.set_title("Mean intensity per slice")
line_mean, = ax_plot.plot(x_idx, slice_mean, lw=2, label="mean")

band_poly = None
if show_mean_std_band:
    band_poly = ax_plot.fill_between(
        x_idx, slice_mean - slice_std, slice_mean + slice_std,
        alpha=0.18, label="mean ± std"
    )

marker_point, = ax_plot.plot([1], [slice_mean[0]], "o", ms=8, color="orange", label="current")
cursor_line = ax_plot.axvline(1, ls="--", lw=1.4, alpha=0.8, color="orange")
peak_line = ax_plot.axvline(peak_idx, ls=":", lw=1.4, color="tab:green", label=f"peak = {peak_idx}")

annot = ax_plot.annotate(f"{slice_mean[0]:.0f}",
                         xy=(1, slice_mean[0]),
                         xytext=(10, 10), textcoords="offset points",
                         bbox=dict(boxstyle="round,pad=0.2", fc="white", alpha=0.8))

ax_plot.set_xlabel("Slice index")
ax_plot.set_ylabel("Mean intensity (a.u.)")
ax_plot.grid(True, alpha=0.3)
ax_plot.legend(loc="best")

ymin, ymax = slice_mean.min(), slice_mean.max()
pad = 0.05 * (ymax - ymin + 1e-9)
ax_plot.set_xlim(1, frames)
ax_plot.set_ylim(ymin - pad, ymax + pad)

# ---------- init & update ----------
def init():
    im.set_data(image_stack[:, :, 0])
    ax_img.set_title(f"{varname} | Slice 1 / {frames}")
    marker_point.set_data([1], [slice_mean[0]])
    cursor_line.set_xdata([1, 1])     
    annot.xy = (1, slice_mean[0])   
    annot.set_text(f"{slice_mean[0]:.0f}")
    return [im, marker_point, cursor_line, annot]

def update(i: int):
    im.set_data(image_stack[:, :, i])
    ax_img.set_title(f"{varname} | Slice {i+1} / {frames}")

    y = slice_mean[i]
    marker_point.set_data([i + 1], [y])
    cursor_line.set_xdata([i + 1, i + 1])  
    annot.xy = (i + 1, y)                 
    annot.set_text(f"{y:.0f}")
    return [im, marker_point, cursor_line, annot]

# ---------- animation ----------
ani = animation.FuncAnimation(
    fig, update,
    init_func=init,
    frames=frames,
    interval=int(1000 / fps),
    blit=False,
    save_count=frames,   
)

ani.save(output_gif, writer="pillow", fps=fps)
plt.close(fig)
print(f" ไฟล์ภาพพร้อมใช้งานแล้ว '{output_gif}' ({frames} frames)")


shape=(201, 201, 61) | ใช้ 61 ระนาบในการทำแอนิเมชัน
ช่วงคอนทราสต์: vmin=0.00, vmax=300000.00


C:\Users\JIRAP\AppData\Local\Temp\ipykernel_3948\2318849135.py:124: UserWarning: You passed in an explicit save_count=61 which is being ignored in favor of frames=61.
  ani = animation.FuncAnimation(


✅ ไฟล์ภาพพร้อมใช้งานแล้ว 'animation_with_cursor.gif' (61 frames)
